## Accuracy Check using IOU

In [ ]:
#Method for calculating accuracy for single test image.

import numpy as np
import cv2 as cv
face_cascade = cv.CascadeClassifier('C:\\Users\\DELL\\Desktop\\Praxis\\Term3\\Capstone\\VideoDetect\\myhaar_p1069n2654s20.xml')
img = cv.imread('C:\\Users\\DELL\\Desktop\\Praxis\\Term3\\Capstone\\VideoDetect\\NB_8.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
boxA = [940,288,1229,594]   #Co-ordinates of image for whch testing is to be done.
boxB=[]
faces = face_cascade.detectMultiScale(gray, 1.3, 5)
if type(faces) == tuple:
    if all(faces):         #If the face is not present in the image then iou = 0.
        iou = 0
for (x,y,w,h) in faces:
    cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    boxB.append(x)
    boxB.append(y)
    boxB.append(w+x)
    boxB.append(h+y)
    iou = bb_intersection_over_union(boxA,boxB)   #Call function to calculate IoU.
    boxB = []
print(iou)
cv.destroyAllWindows()

In [4]:
#Function to calculate IoU.

def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    #print(interArea)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
 
    # return the intersection over union value
    return iou

In [1]:
import pandas as pd

In [12]:
#For IoU calculation for multiple images
import numpy as np
import cv2 as cv
import glob
import os
#face_cascade = cv.CascadeClassifier('C:\\Users\\DELL\\Desktop\\Praxis\\Term3\\Capstone\\VideoDetect\\myhaar_p1069n2654s20.xml')
face_cascade = cv.CascadeClassifier('C:\\Users\\DELL\\Desktop\\Praxis\\Term3\\Capstone\\VideoDetect\\myhaar_1105P_2760N_LF.xml')
count = 0
dicty = {}
for imag in glob.glob("C:\\Users\\DELL\\Desktop\\Praxis\\Term3\\Capstone\\VideoDetect\\For Accuracy Test\\TestImg_30082018\\*.jpg"):
    img= cv.imread(imag)
    imgName = os.listdir("C:\\Users\\DELL\\Desktop\\Praxis\\Term3\\Capstone\\VideoDetect\\For Accuracy Test\\TestImg_30082018")[count] #Get image names of all the files in the directory
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    boxA = getActualFaceCoord(imgName)  #Function which will return the face co-ordinates according to info.txt.
    boxB=[]
    iou = 0
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if type(faces) == tuple:
        if all(faces):        #This line is to check that if the face is not detected then it will be empty. 
            iou = 0           # If the face is not detected, set the iou to 0.
            #print(iou)
            dicty[imgName] = iou
    for (x,y,w,h) in faces:
        cv.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        boxB.append(x)
        boxB.append(y)
        boxB.append(w+x)
        boxB.append(h+y)
        iou = bb_intersection_over_union(boxA,boxB)   #Function to calculate iou.
        #print(iou)
        boxB = []
        dicty[imgName] = iou
        #break
    count += 1
    cv.putText(img, "IoU: {:.4f}".format(iou), (100, 100), cv.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 2)
    cv.imwrite("C:\\Users\\DELL\\Desktop\\Praxis\\Term3\\Capstone\\VideoDetect\\For Accuracy Test\\DetectedImg\\"+imgName, img)
cv.destroyAllWindows()
df = pd.DataFrame(list(dicty.items()),columns=["Image","Accuracy"])
df.to_excel("C:\\Users\\DELL\\Desktop\\Praxis\\Term3\\Capstone\\VideoDetect\\For Accuracy Test\\Detected_Excel\\acc_17092018.xls")

In [2]:
#Function to get actual coordinates from info.txt for test images.

def getActualFaceCoord(imageName):
    fhand = open("C:\\Users\\DELL\\Desktop\\Praxis\\Term3\\Capstone\\VideoDetect\\info.txt")
    boxA = []
    imageName = imageName.replace(".jpg",'')
    for line in fhand:
        if(imageName in line):
            c = 0
            for vals in line.strip().split(' '):  #Need to get only the co-ordinates of the image.
                if c > 1:
                    boxA.append(int(vals))
                c += 1
            break
    w = boxA[2]
    h = boxA[3]
    boxA[2] = boxA[0]+w
    boxA[3] = boxA[1]+h
   
    return boxA